# Using fine-tuned models
---
This notebook is intended to serve as an example of how to communicate and use previously fine-tuned models.The different model names are stored in Models.jsonl.

#### Imports

In [1]:
import openai
from openai import OpenAI
import os
import time

import random
import re
import pickle
import scapy

from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy
from scapy.layers.l2 import Ether, ARP
from scapy.layers.inet import IP, ICMP, TCP
from scapy.layers.http import *

C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:469: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:483: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


#### Definitions

In [3]:
os.environ['OPENAI_API_KEY'] = 'FILL'
client = OpenAI()

tipo ="Conversations"
wanted_Model = "HTTP"
Models_file = 'Models.jsonl'

dataset = "../../data/Conversations/HTTP/HTTP_Conv_Testing.txt"
pkl_saved = "../../data/Fine_Tuned/HTTP/pickle/Test_HTTP_CONV_Fine_Tuned_v2.pkl"
pcap_saved = "../../data/Fine_Tuned/HTTP/pcap/Test_HTTP_CONV_v2_generated_3_5_turbo-instruct_conv.pcap"

system_message = 'You are a new generation traffic generator. \
You are specilized in the HTTP protocol and conversations generation using python and scapy. \
You are especially attentive to variables and different types of traffic.'

### Fine-Tuned models list

In [5]:
with open(Models_file, 'r') as json_file:
    Models_list = json.load(json_file)

print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("                                   Paquetes                                   ")
print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n\n")
print("----------Lista de modelos de paquetes----------")
for key, value in Models_list["Packets"]["Specific"].items():
    print( "\t" + key + " -> " + value)

    
print("----------Problemas de los modelos de paquetes----------")
for key, value in Models_list["Packets"]["Problems"].items():
    print( "\t" + key + " -> " + value)

print("\n\n\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("                               Conversaciones                                 ")
print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n\n")
print("----------Lista de modelos de Conversaciones----------")
for key, value in Models_list["Conversations"]["Specific"].items():
    print( "\t" + key + " -> " + value)

    
print("----------Problemas de los modelos de Conversaciones----------")
for key, value in Models_list["Conversations"]["Problems"].items():
    print( "\t" + key + " -> " + value)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
                                   Paquetes                                   
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


----------Lista de modelos de paquetes----------
	ICMPv1 -> ft:gpt-3.5-turbo-0125:personal:icmpv1:9ELvoktd
	ICMPv2 -> ft:gpt-3.5-turbo-1106:personal:icmpv2:9F2iiDZa
	DNS -> 
	ARP -> ft:gpt-3.5-turbo-1106:personal:arpv1:9FLIvsUk
	ARPv2 -> ft:gpt-3.5-turbo-1106:personal:arpv2:9FPXrE3X
	HTTP -> 
----------Problemas de los modelos de paquetes----------
	ICMPv1 -> Las respuestas se dan con un 	 al principio, por lo tanto no se puede ejecutar. AdemÃ¡s, solo responde a peticiones de 1 solo paquete.
	ICMPv2 -> Sigue generando solo 1 paquete por peticiÃ³n.
	DNS -> 
	ARP -> ConfundÃ­a paquetes aleatorios con ARP gratuitos.
	ARPv2 -> 
	HTTP -> 



++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
                               Conver

### Loading the query dataset

In [7]:
with open(dataset,"r") as f:
    packets_summary = f.read().splitlines()
len(packets_summary)

8

### Loading a pickle file to save the responses 

In [9]:
class response():
    "Stores name and place pairs"
    def __init__(self, name, place):
        self.prompt_summary = name
        self.completion = place

try:
    responses = pickle.load( open(pkl_saved, "rb" ))
    print("Already" + str(len(responses)) + " generated.")
except:
    responses = []
    print("No responses previously generated")

No responses previously generated


### Packet command generation using the fine tuned model

In [16]:
for index in range( len(packets_summary)):
    
    summary = packets_summary[index]
    print("...........................................")
    print( " GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter " +str(index+1) +")")
    
    
    #summaries = random.choices(packets_summary,k=summaries_per_request)
    
    messages=[{"role": "system", "content": system_message},
        {"role": "user", "content": summary}]
    
    print("\n This is the message sent: ")
    print('\t"role": "system", "content" : \n\t\t' + system_message)
    print('\n\t"role": "user", "content" : ' + summary)
    #for summary in summaries:
    #    print("\t\t" + summary)
    
    #Esta es la petición que le mandas al modelo de IA
    completion = client.chat.completions.create(
        model=Models_list[tipo]["Specific"][wanted_Model], 
        messages=messages,
        max_tokens=2600,
        temperature=0.1)

    
    print("...Generated!")
    
    # Guardamos tanto el resumen de los paquetes que solicitamos y lo que devuelve el modelo para estos paquetes
    responses.append(response(messages, completion))
    
    print("The response is: ")
    print(responses[-1].completion.choices[0].message.content)
    
    print("\nNumber of completitions done: " + str(len(responses)))
    
    exec(responses[-1].completion.choices[0].message.content)
    print("\n")
    
    pkt_list = [pkt_1, pkt_2, pkt_3, pkt_4, pkt_5, pkt_6, pkt_7, pkt_8, pkt_9, pkt_10, pkt_11]
    
    # Guardamos los paquetes generados en un pcap
    with open(pcap_saved, "ba+") as f:
        wrpcap(f, pkt_list, append=True)
        
    # Guardamos tanto la petición como la respuesta en un archivo
    pickle.dump(responses, open(pkl_saved, "wb" ) )

...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 1)

 This is the message sent: 
	"role": "system", "content" : 
		You are a new generation traffic generator. You are specilized in the HTTP protocol and conversations generation using python and scapy. You are especially attentive to variables and different types of traffic.

	"role": "user", "content" : Source: IP="10.0.0.200", port=9352, Window: 8192 // Destination: IP="10.0.0.201", port=80, Window: 1384 // Others: Host="http://www.adobe.com/", Path="", Code=200,  Reason_Phrase= "OK"
...Generated!
The response is: 

x,y = random.sample(range(1, 100), 2)
pkt_1= IP(src="10.0.0.200", dst="10.0.0.201")/TCP(sport=9352, dport=80, flags="S", seq=y, window=8192)
time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_1[TCP].payload)+1
pkt_2= IP(src="10.0.0.201", dst="10.0.0.200")/TCP(sport=80, dport=9352, flags="SA", seq=x, ack=y, window=1384)
time.sleep(abs(random.gauss(0, 0.03)))
x += len(pk

In [29]:
print(responses[-1].completion.choices[0].message.content)

scapy.Ether(src="c2:b3:a4:f5'e6'd7", dst="c2:b3:a4:f5:e6:d8")/scapy.ARP(op=2, psrc="172.16.0.11", hwsrc="c2:b3:a4:f5:e6:d7", hwdst="c2:b3:a4:f5:e6:d8",  pdst=socket.inet_ntoa(struct.pack(">I", random.randint(1, 0xffffffff))))


### Packet generation

In [39]:
# Ejecutamos los comandos que devuelve el modelo para generar los paquetes
exec("pkt= " + str(responses[-1].completion.choices[0].message.content))
print(pkt)

b'E\x00\x00\x1c\x00\x01\x00\x006\x01\xb2\x80\x05#\xe2\x88\xc0\xa8*\x0c\x00\x00\xfa\x98\x04V\x01\x11'


In [43]:
#Enviarlos para probar si se envian correctamente
send(pkt)


Sent 1 packets.


### Introduction to a pcap

In [46]:
# Guardamos los paquetes generados en un pcap
with open(pcap_saved, "ba+") as f:
    wrpcap(f, pkt, append=True)